In [2]:
import numpy as np

X = []
Y = []

x_noise = 0.0
y_noise = 0.0

# TODO: randomly occlude large parts of the state input to simulate detection inconsistencies

model_name = "combined_junction_straight"

with open(f"../utils/datasets/{model_name}.csv") as f:
    for i, line in enumerate(f):
        data = line.split(",")
        x = np.array(data[2:], dtype=np.float32)
        y = np.array(data[:2], dtype=np.float32)

        x[x > 1] = 0  # track only
        y = y[1]  # steer only

        grid_size = 25
        x = x.reshape(grid_size, grid_size, 1)  # transform state back to grid

        X.append(x)
        Y.append(y)

X = np.array(X, dtype=np.float32)
Y = np.array(Y, dtype=np.float32)

print(f"Loaded {len(X)} entires")
print(f"entry {x.shape=}, {y.shape=}")

Loaded 29272 entires
entry x.shape=(25, 25, 1), y.shape=()


In [1]:
import keras
print(keras.__version__)

2.13.1


In [3]:
import keras

model = keras.models.Sequential(
    [
        # keras.layers.InputLayer(shape=x.shape),
        # lenet
        keras.layers.Conv2D(6, kernel_size=(5, 5), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(16, kernel_size=(5, 5), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(120, activation="relu"),
        keras.layers.Dense(84, activation="relu"),
        keras.layers.Dense(1),
    ]
)

# TODO: add normal following to all experts' ds

model.compile(
    optimizer="adam",
    loss="mean_squared_error",
)

model.fit(
    X,
    Y,
    epochs=10,
    validation_split=0.2,
    callbacks=[keras.callbacks.EarlyStopping(patience=3)],
)

model.save(f"../controllers/moe_controller/models/{model_name}-new.keras")

Epoch 1/10
732/732 [==============================] - 2s 3ms/step - loss: 0.0220 - val_loss: 0.0383
Epoch 2/10
732/732 [==============================] - 2s 3ms/step - loss: 0.0172 - val_loss: 0.0366
Epoch 3/10
732/732 [==============================] - 2s 3ms/step - loss: 0.0160 - val_loss: 0.0365
Epoch 4/10
732/732 [==============================] - 2s 3ms/step - loss: 0.0153 - val_loss: 0.0361
Epoch 5/10
732/732 [==============================] - 2s 3ms/step - loss: 0.0148 - val_loss: 0.0383
Epoch 6/10
732/732 [==============================] - 2s 3ms/step - loss: 0.0141 - val_loss: 0.0424
Epoch 7/10
732/732 [==============================] - 2s 3ms/step - loss: 0.0136 - val_loss: 0.0384


In [25]:
# time inference speed
import time

start = time.time()
model(X[0:100])
end = time.time()
avg = (end - start) / 100
print(f"avg inference time: {avg*1000:.2f}ms")

avg inference time: 0.11ms
